In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import re

In [9]:
!./ICDF.out

In [10]:
#Creamos lista de DF
DF = pd.DataFrame()
for i in glob.glob('./Results/*.dat'):
    data = pd.read_csv(i, sep = " ")
    data['Iteration'] = int(re.findall(r'\d+',i)[0])
    DF =  pd.concat([DF,data],ignore_index=True)

In [11]:
#Sacamos el N mas frecuente
N_most_frequent = DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]
N = N_most_frequent["#Autos"].mode()[0]
iterations = N_most_frequent[N_most_frequent['#Autos'] == N].Iteration.unique()

In [12]:
DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]['#Autos'].value_counts()

614    934
618    878
640    807
494    553
466    515
534    502
583    474
595    468
586    458
571    454
605    450
646    430
648    415
638    408
654    392
695    388
651    383
598    362
637    361
652    331
622     37
Name: #Autos, dtype: int64

In [13]:
#Seleccionamos todos los procesos que contengan al N mas frecuente
DF_ = DF[DF.Iteration.isin(iterations)]

In [15]:
pd.pivot_table(DF_, index = 'Iteration', values = 'Delta_t', aggfunc = [np.mean,np.std, np.median])

,mean,std,median
,Delta_t,Delta_t,Delta_t
Iteration,,,
2620,5.878588,9.533488,3.39784
2621,5.878588,9.533488,3.39784
2622,5.878588,9.533488,3.39784
2623,5.878588,9.533488,3.39784
2624,5.878588,9.533488,3.39784
...,...,...,...
5737,5.876568,7.278710,3.61761
5738,5.876568,7.278710,3.61761
